In [519]:
import numpy as np

class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False, anti_windup_backcalc=True,
                 beta=1, linearity=1, alpha=1, deadband=0):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        if self.itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.dif = do_ifactor
        self.awb = anti_windup_backcalc

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0
        self.b = beta
        self.l = linearity
        self.a = alpha
        self.deadband = deadband

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        #err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        err_for_pgain = (sp-pv) * self.b
        uk0 = self.pgain * err_for_pgain
        self.bump = op - uk0
        self.last_pv = pv
        
    def auto_to_auto(self, pv, op):
        self.man_to_auto(pv, pv, op)
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        
    
    def step(self, pv, sp, dt=1):
        err = sp - pv
        if err > 0:
            err -= self.deadband
        else:
            err += self.deadband
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        # SP_Range is taken to be 100 for both of the below lines
        # since range is calculated as +/- middle (?)
        # so -100-100 is actually range of 100 according to LV (??)
        #err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        err_for_pgain = err * self.b
        if self.dif:
            ierr *= 1 / (1 + err * err * 10 / (100*100))  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err_for_pgain
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain * self.a
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui * self.pgain
        self.Uk = Uk
        self.Ud = Ud * self.pgain
        self.Up = Up
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc and self.awb:
            ierr = self.itime / self.pgain * (am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

In [520]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [521]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]
    
class DelaySink():
    def __init__(self, delay, initial, pc_at_delay=0.95, dt=1):
        assert 0 < pc_at_delay <= 1, "don't be a retard"
        if delay:
            self.df = 1 - (1-pc_at_delay)**(dt/delay)
            self.sink = initial / self.df  - initial # fill the sink
        else:
            self.df = 0
            self.sink = 0
        self.dt = dt
        if self.sink < 0:
            self.sink = 0
        
    def delay(self, op):
        self.sink += op * self.dt
        dNdt = self.sink*self.df
        self.sink -= dNdt
        return dNdt

In [522]:
TOTAL_RVOLUME = {
    3: 4,
    15: 17,
    80: 90,
}

class DOProcess():
    """ DO Process
    Uses cascaded calculation model,
    First using HeadspaceProcess to estimate
    headspace gas concentrations, and using
    the result to calculate change in PV. 
    """
    # process gain constant units of  
    # % per delta% per hour 
    # dpv/dt = k*([O2]/0.2095 * 100 - DOPV)
    # where [02] is volume fraction, DOPV is in %
    default_k = 0.1306
    
    # consumption rate constant in units of % per hour
    # presumably, consumption rate is "Constant"
    # within approximation, as long as cellular conditions
    # don't transition between aerobic and anaerobic
    default_c = 0
    
    # volume fraction of CO2, N2, O2 in air
    # note the remaining 1% is trace gasses
    AIR_CNO = (0.0004, 0.7809, 0.2095)
    
    def __init__(self, main_gas=1, initial_pv=60, initial_cno=AIR_CNO, reactor_size=80, volume=55, delay=0):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial_pv).cycle
        self.k = self.default_k / 3600  # in seconds
        self.c = self.default_c / 3600  # in seconds
        self._reactor_size = reactor_size
        self._volume = volume
        self.main_gas = main_gas
    
        hs_volume = TOTAL_RVOLUME[reactor_size] - volume
        c, n, o = initial_cno
        self.hp = HeadspaceProcess(hs_volume, c, n, o)
        
    @property
    def volume(self): 
        return self._volume
    
    @volume.setter
    def volume(self, v):
        self._volume = v
        self.hp.vol = TOTAL_RVOLUME[self._reactor_size] - v

    def step(self, pv, co2_req, n2_req, o2_req, air_req):
        self.hp.calc_gas(self.main_gas, co2_req, n2_req, o2_req, air_req)
        o2 = self.tdelay(self.hp.o2A)
        pvarg = (o2 * 100 / 0.2095 - pv)
        dpv = self.k * pvarg
        pv += dpv
        pv -= self.c
        if pv < 0:
            return 0
        return pv      
    

class HeadspaceProcess():
    def __init__(self, vol, co2A=0, n2A=0.78, o2A=0.21):
        """ 
        vol: volume headspace (L)
        main_gas: main gas flow rate (L/min)
        co2: CO2 request (%)
        n2: N2 request (%)
        
        Calculate headspace concentration. DT = 1 second. 
        """
        self.vol = vol
        self.co2A = co2A
        self.n2A = n2A
        self.o2A = o2A
        self.airA = 1 - (co2A+n2A+o2A)
        
    def step(self, main_gas, co2_req, n2_req, o2_req, air_req):
        self.calc_gas(main_gas, co2_req, n2_req, o2_req, air_req)
        return self.o2A
    
    def calc_gas(self, main_gas, co2_req, n2_req, o2_req, air_req):
        """ 
        Calculate new headspace gas concentration.
        Assumptions:
            - Mixing time is 0 seconds, i.e. gas mixture is always perfectly mixed
            - Gas lost from headspace is always equal to gas entering headspace
            - Concentration of individual gasses lost match headspace concentration of gasses. 
            - Mass transfer coefficient of Air <-> liquid k <<< Mg (no volume lost due to absorption to liquid)
            - Air O2: 20.95%, N2: 78.09%, CO2=0.04%, other = 1-(o2+n2+co2)
        """
        mg = main_gas / 60
        
        # volume of gas lost
        co2L = mg * self.co2A
        n2L = mg * self.n2A
        o2L = mg * self.o2A
        airL = mg * self.airA
        
        # volume of gas gained
        # airG is non-corrected volume of air added
        co2G = mg * co2_req
        n2G = mg * n2_req
        o2G = mg * o2_req
        airG = mg * air_req
        
        # air contribution to gases
        # airG is corrected to reflect
        # the volume of non-tracked gases
        # ie the 1% of trace elements or so
        co2G += 0.0004 * airG
        o2G += 0.2095 * airG
        n2G += 0.7809 * airG
        airG = .0092 * airG
        
        # new volume of each gas
        co2V = co2G - co2L + self.co2A * self.vol
        n2V = n2G - n2L + self.n2A * self.vol
        o2V = o2G - o2L + self.o2A * self.vol
        airV = airG - airL + self.airA * self.vol
        
        # finally, new percentages
        self.co2A = co2V / self.vol
        self.n2A = n2V / self.vol
        self.o2A = o2V / self.vol
        self.airA = airV / self.vol
        
        # Sanity checks
        # Errors should be on the order of 
        # floating point rounding error ~1e-14
        self._err_ = self.vol - sum((co2V, n2V, o2V, airV))
        self._err2_ = 1 - (self.co2A + self.n2A + self.o2A + self.airA)
        self._err3_ = (co2L+n2L+o2L+airL) - (co2G + n2G + o2G + airG)
            

In [523]:
class GasController():
    def __init__(self, co2mfcmax, n2mfcmax, o2mfcmax, airmfcmax):
        self.cm = co2mfcmax
        self.nm = n2mfcmax
        self.om = o2mfcmax
        self.am = airmfcmax
   
    def request(self, main_gas, co2_req, n2_req, o2_req):

        co2max = self.cm / main_gas
        co2pc = co2max if co2_req > co2max else co2_req

        o2_req_max = 1 - co2pc
        if o2_req > o2_req_max: o2_req = o2_req_max
        o2max = self.om / main_gas
        o2pc = o2max if o2_req > o2max else o2_req
        
        n2_req_max = 1 - co2pc - o2pc
        if n2_req > n2_req_max: n2_req = n2_req_max
        n2max = self.nm / main_gas
        n2pc = n2max if n2_req > n2max else n2_req
        
        apc = 1 - co2pc - o2pc - n2pc
        
        return co2pc, n2pc, o2pc, apc

In [524]:
def _round(dig, *args):
    rv = []
    for a in args:
        rv.append(round(a, dig))
    return rv
def test_gas_controller():
    ctrl = GasController(1, 10, 2, 10)
    
    c,o,n = (0.8, 0.3, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(0.5, c,n,o))
    assert rc == 0.8, rc
    assert ro == 0.2, ro
    assert rn == 0, rn
    assert ra == 0, ra
    
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0.2, ro
    assert rn == 0.7, rn
    assert ra == 0, ra
    
    c,o,n = (0.8, 0, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0, ro
    assert rn == 0.9, rn
    assert ra == 0, ra
    
    c,o,n = (0.07, 0.1, 0.3)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.07, rc
    assert ro == 0.1, ro
    assert rn == 0.3, rn
    assert ra == 0.53, ra
    
    for mg in (0.5, 1, 5, 10):
        for c in 0.1, 0.5, 0.8, 1:
            for o in 0.1, 0.5, 0.8, 1:
                for n in 0.1, 0.5, 0.8, 1:
                    args = (mg, c, n, o)
                    ec = round(min(1/mg, c), 5)
                    eo = round(min(2/mg, (1-ec), o),5)
                    en = round(min(1-ec-eo, n),5)
                    ea = round(1-ec-eo-en,5)
                    rc, rn, ro, ra = _round(5, *ctrl.request(*args))
                    assert rc == ec, (rc, ec, args)
                    assert ro == eo, (ro, eo, args, (rc, rn, ro, ra))
                    assert rn == en, (rn, en, args)
                    assert ra >= 0 and ra == ea, (ra, ea, args)
    
test_gas_controller()

In [525]:
ypv = []
yco2 = []
yn2 = []
yo2 = []
err = []
err2 = []
err3 = []
pv = 60
mg = 1
co2r = 0.07
n2r = 0.10
o2r = 0.20
proc = DOProcess(mg, pv, DOProcess.AIR_CNO, 80, 55)
seconds = 72000
for _ in range(seconds):
    pv = proc.step(pv, co2r, n2r, o2r)
    ypv.append(pv)
    yco2.append(proc.hp.co2A)
    yn2.append(proc.hp.n2A)
    yo2.append(proc.hp.o2A)
    err.append(proc.hp._err_)
    err2.append(proc.hp._err2_)
    err3.append(proc.hp._err3_)
    
for _ in range(seconds):
    pv = proc.step(pv, 0.14, 0.3, 0.05)
    ypv.append(pv)
    yco2.append(proc.hp.co2A)
    yn2.append(proc.hp.n2A)
    yo2.append(proc.hp.o2A)
    err.append(proc.hp._err_)
    err2.append(proc.hp._err2_)
    err3.append(proc.hp._err3_)
    
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
fm = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))
plt.close()
x = np.arange(len(ypv)) / 3600


def lowpass(data, alpha):
    y = np.zeros_like(data)
    ys = data[0]
    for i, d in enumerate(data):
        ys += alpha * (d - ys)
        y[i] = ys
    return y
# err = lowpass(err, 1/1000)
# err2 = lowpass(err2, 1/1000)
# ax1 = plt.gca()
# plt.plot(x, err, "blue", ls="-", label="err")
# plt.plot(x, err2*100, "green", ls="-", label="err2")
# plt.gca().twinx().plot(x, err3, "purple", ls="-", label="PV")

plt.plot(x, ypv, "blue", ls="-", label="PV")
ax2 = plt.gca().twinx()
ax2.plot(x, yco2, "purple", ls="-", label="CO2")
ax2.plot(x, yo2, "red", ls="-", label="O2")
ax2.plot(x, yn2, "green", ls="-", label="N2")


plt.legend(loc="center left")
plt.gca().yaxis.set_major_formatter(fm)

air_req = 1 - (o2r+n2r+co2r)
o2 = o2r + air_req * 0.2095
n2 = n2r + air_req * 0.7809
co2 = co2r + air_req * 0.0004
# ax2.axhline(y=o2, ls="--", color="black")
# ax2.axhline(y=n2, ls="--", color="black")
# ax2.axhline(y=co2, ls="--", color="black")
# plt.axvline(x=seconds/3600, ls="--", color="black")
plt.show()

TypeError: step() missing 1 required positional argument: 'air_req'

In [ ]:
class OptionCategory():
    def __iter__(self):
        d = self.__class__.__dict__.copy()
        d.update(self.__dict__)
        ob = ((k, v) for k,v in d.items() if k[0] != "_" and k[-1]!="_")
        return iter(ob)
    
    def __repr__(self):
        buf = []
        for k, v in self:
            if isinstance(v, OptionCategory):
                v = "<More Options...>"
            buf.append("%s: %s" %(k,v))
        return "\n".join(buf)
    
    def __init__(self):
        self.__dict__.update(self.__class__.__dict__)
    
    def __setattr__(self, k, v):
        if k not in self.__dict__:
            raise AttributeError(k)
        object.__setattr__(self, k, v)

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    deadband = 1
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10

class SimOps(OptionCategory):
    
    o2_pid = PIDOps()
    o2_pid.p = 5
    o2_pid.i = 5
    o2_pid.d = 0
    o2_pid.amax = 100
    o2_pid.amin = 0
    o2_pid.beta = 1
    o2_pid.linearity = 1
    o2_pid.alpha = 1
    
    n2_pid = PIDOps()
    n2_pid.p = 5
    n2_pid.i = 5
    n2_pid.d = 0
    n2_pid.amax = 0
    n2_pid.amin = 0
    n2_pid.beta = 1
    n2_pid.linearity = 1
    n2_pid.alpha = 1
    
    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 2
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    delay = 0
    end = 10000
    initial_actual_cno = DOProcess.AIR_CNO
    initial_request_cno = (0.07, 0, 0)
    initial_pv = 90
    set_point = 40
    k = None
    c = None
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80


In [526]:

def do_sim(ops):
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp, o2_req*100)
        n2_pid.man_to_auto(pv, sp, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    if ops.k is not None:
        proc.k = ops.k
    if ops.c is not None:
        proc.c = ops.c
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    end = ops.end
    data = [(t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a)]
    while True:
        t += 1
        o2_req = o2_pid.step(pv, sp) / 100
        n2_req = n2_pid.step(pv, sp) / 100
        co2_req, o2_req, n2_req, air_req = ctrl.request(mg, co2_req, o2_req, n2_req)
        pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)
        co2a = proc.hp.co2A
        n2a = proc.hp.n2A
        o2a = proc.hp.o2A
        data.append((t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a))
        if t >= end:
            break
    return data

In [527]:
ops = SimOps()
ops.delay = 0
ops.end = 10000
ops.initial_actual_cno = DOProcess.AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 100
ops.set_point = 90
ops.k = None
ops.c = None
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55

In [528]:
# O2 pid
ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.d = 0
ops.o2_pid.alpha = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.linearity = 1
ops.o2_pid.beta = 1

In [529]:
# N2 PID
ops.n2_pid.p = -5
ops.n2_pid.i = 5
ops.n2_pid.d = 0
ops.n2_pid.amax = 100
ops.n2_pid.amin = 0
ops.n2_pid.alpha = 1
ops.n2_pid.beta = 1
ops.n2_pid.linearity = 1

In [530]:
# data is (t, pv, mg, o2_req, n2_req, co2_req, co2a, n2a, o2a)
seconds = 1
minutes = seconds * 60
hours = minutes * 60
days = hours * 24
ops.end = 1 * days
sp = 50

ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 1

ops.n2_pid.p = -5
ops.n2_pid.i = 10
ops.n2_pid.amax = 100
ops.o2_pid.deadband = 1

ops.initial_pv = 50
ops.set_point = sp
ops.c = 0.000

ops.initial_request_cno = (0.00, 0, 0)
print("Beginning simulation")

data = do_sim(ops)
x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a = list(zip(*data))
plt.close()
print("Plotting Data")

wm=plt.get_current_fig_manager()
wm.window.attributes('-topmost', 1)
wm.window.attributes('-topmost', 0)
# h = wm.window.winfo_height()
# w = wm.window.winfo_width()
# wm.window.geometry("%sx%s+%s+%s"%(w,h,800, 150))

step = 100
xs = x[::step]
xs = np.array(xs) / hours

ax1 = plt.gca()
ax1.plot(xs, pv[::step], "blue", ls="-", label="PV")
ax2 = ax1.twinx()
ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

ax1.yaxis.set_major_formatter(fm1)
ax2.yaxis.set_major_formatter(fm2)
ax2.set_ylim((0, 1.1))
#ax1.set_ylim((sp-20, sp+20))
#ax1.set_xlim((40, 45))
ax1.grid()
plt.legend()

Beginning simulation
Plotting Data


In [456]:
x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a = list(zip(*data))
plt.close()
print("Plotting Data")

wm=plt.get_current_fig_manager()
wm.window.attributes('-topmost', 1)
wm.window.attributes('-topmost', 0)
# h = wm.window.winfo_height()
# w = wm.window.winfo_width()
# wm.window.geometry("%sx%s+%s+%s"%(w,h,800, 150))

step = 100
xs = x[::step]
xs = np.array(xs) / hours

ax1 = plt.gca()
ax1.plot(xs, pv[::step], "blue", ls="-", label="PV")
ax2 = ax1.twinx()
ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

ax1.yaxis.set_major_formatter(fm1)
ax2.yaxis.set_major_formatter(fm2)
ax2.set_ylim((0, 1.1))
#ax1.set_ylim((sp-20, sp+20))
#ax1.set_xlim((40, 45))
ax1.grid()
plt.legend()

Plotting Data


In [454]:
plt.grid()
ax1.grid()

In [72]:
def paste(cells, data, offset=0, headers=None):
    headers = headers or [("T", "PV", "OP")]
    ws = cells.Parent
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, len(headers)+offset)).Value = headers
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        bottomleft = topleft.Offset(len(data), 1)
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data
        chart = ws.ChartObjects(1).Chart
        s1 = chart.SeriesCollection(1)
        s1.XValues = cells.Range(topleft, bottomleft)
        s1.Values = cells.Range(topleft.Offset(1,2), bottomleft.Offset(1,2))
        s1.Name = cells(1,1).Offset(1, 2).Value
        
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [73]:
# def format_chart(ws, cells, topleft, bottomleft, 
#                  chart, gases, offset2, scnd=False):
#     for i, g in enumerate(gases):
#         try:
#             s = chart.SeriesCollection(i+2)
#         except com_error:
#             s = chart.SeriesCollection().NewSeries()
#         s.XValues = cells.Range(topleft, bottomleft)
#         ytop = topleft.Offset(1, i+offset2)
#         ybot = bottomleft.Offset(1, i+offset2)
#         yrng = cells.Range(ytop, ybot)
#         yrng.NumberFormat = "0.00%"
#         s.Values = yrng
#         s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+4).Address
#         if scnd:
#             s.AxisGroup = xlc.xlSecondary
            

def paste2(cells, data, offset=0, headers=None):
    headers = headers or [("T", "PV", "OP")]
    ws = cells.Parent
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, len(headers)+offset)).Value = headers
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        bottomleft = topleft.Offset(len(data), 1)
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data

        # pv
        try:
            chart = ws.ChartObjects(1).Chart
        except com_error:
            chart = CreateChart(ws, Left=1, Top=1, Width=500, Height=220)
        try:
            s1 = chart.SeriesCollection(1)
        except com_error:
            s1 = chart.SeriesCollection().NewSeries()
        chart = ws.ChartObjects(1).Chart
        chart.HasTitle=True
        chart.ChartTitle.Text = "DOPV(%)"
        s1 = chart.SeriesCollection(1)
        s1.XValues = cells.Range(topleft, bottomleft)
        ytop = topleft.Offset(1,2)
        ybot = bottomleft.Offset(1,2)
        s1.Values = cells.Range(ytop, ybot)
        s1.Name = "=" + "'%s'!"%ws.Name+ cells(1,2).Address
        
        # gas flows
        gases = "CO2 N2 O2".split()
        for i, g in enumerate(gases):
            try:
                s = chart.SeriesCollection(i+2)
            except com_error:
                s = chart.SeriesCollection().NewSeries()
            s.XValues = cells.Range(topleft, bottomleft)
            ytop = topleft.Offset(1, i+4)
            ybot = bottomleft.Offset(1, i+4)
            yrng = cells.Range(ytop, ybot)
            yrng.NumberFormat = "0.00%"
            s.Values = yrng
            s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+4).Address
        # for some reason, need to loop again for this
        for i in range(2, chart.SeriesCollection().Count+1):
            s1 = chart.SeriesCollection(i)
            s1.AxisGroup = xlc.xlSecondary
            s1.AxisGroup = xlc.xlSecondary
        
        # second chart, actual O2 concentrations
        try:
            chart2 = ws.ChartObjects(2).Chart
            delete=False
            start = 1
        except com_error:
            t = chart.Parent.Top
            h = chart.Parent.Height
            chart2 = CreateChart(ws, Left=1, Top=t + h + 20, Width=500)
            chart.ChartType = xlc.xlXYScatter
            s1 = chart2.SeriesCollection().NewSeries()
            delete = True
            start = 2
        for i, g in enumerate(gases, start):
            try:
                s = chart2.SeriesCollection(i)
            except com_error:
                s = chart2.SeriesCollection().NewSeries()
            s.XValues = cells.Range(topleft, bottomleft)
            ytop = topleft.Offset(1, i+7-start)
            ybot = bottomleft.Offset(1, i+7-start)
            yrng = cells.Range(ytop, ybot)
            yrng.NumberFormat = "0.00%"
            s.Values = yrng
            s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+7-start).Address
        if delete:
            s1.Delete()
        
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [74]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [80]:
ops.end=100000
ops.o2_pid.p = 1
ops.o2_pid.i = 0.1
ops.set_point = 50
ops.n2_pid.p = 0
ops.n2_pid.i = 0
ops.n2_pid.amax = 0
ops.c = 0.002
step = 50
with screen_lock(xl):
    clear(cells, data[::step], 0)
    data = do_sim(ops)
    headers = "Time(sec), DOPV(%), Main Gas, CO2 Req, N2 Req, " \
                "O2 Req, CO2 Headspace, N2 Headspace, O2 Headspace"
    headers = headers.split(", ")
    paste2(cells, data[::step], 0, headers)

Setting c 0.002


In [39]:
xl=Excel()
wb = xl.RecentFiles(1).Open()

In [43]:
n2data = xl.Selection.Value2

((41214.4224537037, 0.0),
 (41214.506261574075, 0.0),
 (41214.547939814816, 0.0),
 (41214.58960648148, 0.0),
 (41214.63128472222, 0.0),
 (41214.672951388886, 0.0),
 (41214.71462962963, 0.0),
 (41214.75630787037, 0.0),
 (41214.79798611111, 0.0),
 (41214.83966435185, 0.0))

In [394]:
import json
def getdict(ob):
    return {k:v for k, v in vars(ob).items() if k[0] != "_" and not isinstance(v, OptionCategory)}
d = getdict(SimOps)
d.update(getdict(ops))
d['mfcs'] = getdict(ops.mfcs)
d['o2_pid'] = getdict(ops.o2_pid)
d['n2_pid'] = getdict(ops.n2_pid)
# print(json.dumps(d, indent=4))
s = json.dumps(d, indent=4)
s = "ops = " + s
import clipboard
clipboard.copy(s)
#print(s)

ops2 = {
    "o2_pid": {
        "p": 2,
        "i": 10,
        "d": 0,
        "amax": 100,
        "amin": 0
        "alpha": 0,
        "beta": 1,
        "linearity": 1,
    },
    "n2_pid": {
        "p": -5,
        "i": 5,
        "d": 0,
        "amax": 100,
        "amin": 0
        "alpha": 1,
        "beta": 1,
        "linearity": 1,
    },
    "mfcs": {
        "co2_max": 1
        "n2_max": 10,
        "o2_max": 2,
        "air_max": 10,
    },
    "delay": 0,
    "end": 10000,
    "initial_actual_cno": [0.0004, 0.7809, 0.2095],
    "initial_request_cno": [0.07, 0, 0],
    "initial_pv": 100,
    "set_point": 90,
    "k": null,
    "c": null,
    "mode": "o2a"
    "main_gas": 1.0,
    "reactor_size": 80,
    "reactor_volume": 55,
}

SyntaxError: invalid syntax (<ipython-input-394-6c53dce9943f>, line 23)